# ETL example

In [7]:
import sqlite3
import pandas as pd

In [14]:
# Создаем коннектор к файлу нашей базы данных. 
# Само собой, файлик Базы Данных должен лежать рядом с файлом блокнота (который вы сейчас читаете). 
con = sqlite3.connect('DBname_hr.db')

In [15]:
# Движок СУБД не знает заранее какой объем данных вы будете хранить. 
# Поэтому для доступа к ним он использует "курсор" - некую сущность, которая двигается по таблице.
# Можно представить его в виде скользящего по таблице окна. 
# Перед началом работы его надо инициализировать

cur = con.cursor()

In [16]:
# Создаем таблицу

cur.execute("""CREATE TABLE IF NOT EXISTS employees (
                contact_id INTEGER PRIMARY KEY,
                first_name TEXT NOT NULL,
                last_name TEXT NOT NULL,
                email TEXT NOT NULL UNIQUE,
                phone TEXT
                                        );""").fetchall()

[]

In [17]:
# При необходимости можно жестко удалить таблицу целиоком вместе с данными и структурой 
# cur.execute("""DROP TABLE employees ;""").fetchall()

In [18]:
cur.execute("""INSERT INTO employees (contact_id, first_name, last_name, email)
                        VALUES
                        (1, 'Bud', 'Powell', 'test@test.ru'), 
                        (2, 'Jan', 'Tax', 'test1@test.ru'),
                        (3, 'Max', 'Roberts', 'test2@test.ru'),
                        (4, 'Jannet', 'Wood', 'test3@test.ru')
                        ;""").fetchall()

[]

In [19]:
cur.execute("""INSERT INTO employees (contact_id, first_name, last_name, email)
                        VALUES
                        (5, 'Anjela', 'Rowell', 'test6@test.ru')
                        ;""").fetchall()

[]

In [20]:
# Здесь Python передает предложение SELECT в СУБД (в данном случае это библиотечка sqlite3) и выполняет его, а возвращает набор данных из таблицы (результат работы SELECT)

cur.execute('SELECT * FROM employees').fetchall()

[(1, 'Bud', 'Powell', 'test@test.ru', None),
 (2, 'Jan', 'Tax', 'test1@test.ru', None),
 (3, 'Max', 'Roberts', 'test2@test.ru', None),
 (4, 'Jannet', 'Wood', 'test3@test.ru', None),
 (5, 'Anjela', 'Rowell', 'test6@test.ru', None)]

In [21]:
result = pd.DataFrame(data=cur.execute('SELECT * FROM employees').fetchall())

In [22]:
result.head()

,0,1,2,3,4
0,1,Bud,Powell,test@test.ru,None
1,2,Jan,Tax,test1@test.ru,None
2,3,Max,Roberts,test2@test.ru,None
3,4,Jannet,Wood,test3@test.ru,None
4,5,Anjela,Rowell,test6@test.ru,None


In [23]:
# from datetime import datetime, timedelta
import httplib2

from oauth2client.service_account import ServiceAccountCredentials
import apiclient.discovery

# CREDENTIALS TO SERVICE ACC
CREDSTOSERVICE = 'credentials.json'

In [24]:
spreadsheet_id = '1BIJukq0TZt0RBi5JvjR3WGk3To1T-uZ3s3gNgfsydog'

In [25]:
class Reporting:

    def __init__(self):
        pass

    def auth(self):
        credentials = ServiceAccountCredentials.from_json_keyfile_name(
            CREDSTOSERVICE,
            ['https://www.googleapis.com/auth/spreadsheets',
             'https://www.googleapis.com/auth/drive'])
        httpauth = credentials.authorize(httplib2.Http())
        service = apiclient.discovery.build('sheets', 'v4', http=httpauth)

        return service

    @staticmethod
    def put_values(self):
        values = service.spreadsheets().values().batchUpdate(
            spreadsheetId=spreadsheet_id,
            body={
                "valueInputOption": "USER_ENTERED",
                "data": [
                    {"range": "Sheet1!A1:P",
                     "majorDimension":"ROWS",
                     "values": result}
                ]
            }
        ).execute()


In [26]:
r = Reporting()

In [27]:
# Вызовем авторизацию 
service = r.auth()

In [28]:
# Объект перед отправкой по сети должен поддерживать сериализацию. 
# К сожалению pd DataFrame ее не поддерживает, но мы можем извлечь значения

result = result.values.tolist()

In [29]:
# вызов метода put_values вставит объект result в нужную гугл-таблицу
r.put_values(r)

https://docs.google.com/spreadsheets/d/1BIJukq0TZt0RBi5JvjR3WGk3To1T-uZ3s3gNgfsydog/edit?usp=sharing